In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, ALL, MATCH
import json

import dash_bootstrap_components as dbc
import dash_html_components as html
from datetime import datetime as dt
import pandas as pd
import numpy as np
import time
import random
import plotly.express as px

In [ ]:
df = pd.read_json('C:/Users/Utilisateur/Desktop/Cours/M1/projet inter-promo/groupe 7/donnees/df_final.json')

In [ ]:
def nettoyage(themes_extrait):
    #Création de la liste de sortie
    themes_nettoyes=[]
    #Pour chaque champ, on enlève les caractères 'parasite', on ajoute le résultat dans la liste de sortie
    for i in range(0,len(themes_extrait)):
        if ',' in themes_extrait[i]:
            for j in themes_extrait[i].split(','):
                themes_nettoyes.append(j.strip(" '\"[]#"))
        else:
            themes_nettoyes.append(themes_extrait[i].strip(" '\"[]#"))
    #On enlève les doublons
    themes_nettoyes=list(np.unique(themes_nettoyes))
    themes_nettoyes.remove('')
    return themes_nettoyes

def traitement_themes():
    #Import des la liste des thèmes
    themes=list(df['art_tag'].dropna())
    #On enlève les doublons
#     themes_uniques=list(np.unique(themes))
#     #Appel de la fonction extraction
#     themes_extrait=extraction(themes_uniques)
    #Appel de la fonction nettoyage
    themes_finaux=nettoyage(themes)
    #Retourne les thèmes
    return themes_finaux

def Importation(name: str):
    '''Documentation
    Parameters: name refert to the file name
    output: DataFrame
    '''
    return pd.read_excel(name)

#Function to create language options for the dropdown
def get_lang_options(articles):
    """Documentation
    
    parameters:
        articles: path to the database
    
    attributes:
        list_lang: 'lang' field with no errors and duplicates
        dict: dictionary with the languages found in the database
        
    output:
        lang_options: list of dictionaries for dropdowns
    """
    list_lang=articles['art_lang'].dropna().unique()
    #Declaration of lang_options. It needs to be declared in order to use the append method.
    lang_options=[]
    #This 'for' is to put each language in the dictionary and add it to the list 'lang_options'.
    for n in range(0,len(list_lang)):
        dict={'label':list_lang[n],'value':list_lang[n]}
        lang_options.append(dict)
    return lang_options

#Fonction qui estime le temps de lecture
def count_words(string:str):
    """ 
    Parameters:
        string: We take the words of the article as input
    Attributes:
        string1: it allows to remove the beginning and end space
        count: we initialize count to 1, count will be our word counter
        second: we know that we read 275 words in 1 minute, so we calculate the number of seconds it would take to read the article.
        minute: we convert the number of seconds into minutes
        second2: we make the difference between a minute and the whole part of a minute, which is multiplied by 60. This gives us the remaining seconds
    Out:
        out1: maximum estimated reading time in minutes

    """
    string1=str(string).strip()
    count=1
    for i in string1:
        # a for is used to count the number of words in the article
        if i==" ":
            count+=1
    second2=(count*60)/275
    minute=second/60
    second2=(minute-math.floor(minute))*60
    if second2>0:
        return ("temps maximum de lecture estime a",math.floor(minute)+1," min ")
    else:
        return ("temps maximum de lecture estime a",math.floor(minute)," min ")

#Function to create themes options for the dropdown
def get_cat_options(articles):
    """Documentation
    
    parameters:
        articles: path to the database
    
    attributes:
        list_cat: 'themes' field
        dict: dictionary with the languages found in the database
        
    output:
        cat_options: list of dictionaries for dropdowns
    """
    list_cat=articles['src_name'].dropna().unique()
    #Declaration of cat_options. It needs to be declared in order to use the append method.
    cat_options=[]
    #This 'for' is to put each language in the dictionary and add it to the list 'lang_options'.
    for n in range(0,len(list_cat)):
        dict={'label':list_cat[n],'value':list_cat[n]}
        cat_options.append(dict)
    return cat_options


def get_min_max_dates(articles):
    """Documentation
    
    Parameters:
        articles: we take dataframe type articles as inputs
        
    Attributes:
        date: we retrieve the column art_date where the dates are stored

    Out:
        min_date: the minimum date of the column is retrieved
        max_date: the maximum date of the column is retrieved
    """
    date = articles[['art_date']]
    min_date = min(date['art_date'])
    max_date = max(date['art_date'])
    return (min_date,max_date)

def plot_pie_chart(articles, variable : str , pie_title : str): 
    """Documentation
    
    Parameters:
    
    articles : DataFrame used to plot
    variable : Variable used to compare articles
    pie_title: Title given to the plot
    
    Out: figure 
    """
    articles[variable].fillna("Unknown", inplace = True)
    grp_variable = articles.groupby(variable)[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
    fig = px.pie(grp_variable, values='nb_articles', names=variable, title=pie_title)
    return fig


#graphs
df['art_lang'].fillna("Unknown", inplace = True)
grp_languages = df.groupby('art_lang')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
fig_lang = plot_pie_chart(df,'art_lang','Languages')


df2=df.groupby('src_name')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
fig_themes = px.histogram(df2, x="src_name", y='nb_articles')

In [85]:
# Since we're adding callbacks to elements that don't exist in the app.layout,
# Dash will raise an exception to warn us that we might be
# doing something wrong.
# In this case, we're adding the elements through a callback, so we can ignore
# the exception.

app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

df = df.dropna(subset=['art_title', 'art_url','art_content','src_img','art_tag','art_extract_datetime']).reset_index(drop=True)


tags = traitement_themes()
dic = {}
for i in df['art_tag']:
    if i is None :
        continue
    else :
        for j in tags:
            if j in i:
                if j in dic:
                    dic[j] += 1
                else :
                    dic[j] = 1

top_tags = sorted(dic.items(),key = lambda t:t[1])[-12:-2]

tags_index = 11

# Search Bar

search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(id = "username",type="search", placeholder="Search",debounce=True)),
        dbc.Col(
            dbc.Button("Search", id = "submit-button",color="primary", className="ml-2",href="/"),
            width="auto",
        ),
    ],
    no_gutters=True,
    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

# Nav Bar

navbar = dbc.Navbar(
    [
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(dbc.NavbarBrand("Berger-levrault", className="ml-2")),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://www.berger-levrault.com/fr/",
        ),
        dbc.NavLink("Article", active=True, href="/"),
        dbc.NavLink("Statistiques", href="/stats"),
        dbc.NavLink("Stories",href="/stories"),
        dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(search_bar, id="navbar-collapse", navbar=True),
    ],
    color="dark",
    dark=True,
    fixed = "top",
    sticky = "top",
)


# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "margin-left": "2rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
    "width":"auto",
    "background-color": "#f8f9fa",
}

#Side bar

sidebar = html.Div(
    [
        html.H2("Home"),
        html.Hr(),
        html.Div([
        dbc.Col(
        [
            # -------TOUS LES DROPDOWNS + DATE PICKER ---------
            # gerer fonction du boutton voir components dash (tuto)
            dbc.Col(html.Div([html.Label('Number of article par page'), dcc.Input(
                    id = 'number',
                    type= "number",
                    value = 10,
                    min = 1,
                    max = len(df['art_title']),
                    )])),
            dbc.Col(html.Div([html.Label('Language'),
                              dcc.Dropdown(id='lang_filt',
                    options=get_lang_options(df),
                    multi=True
                    )]), width='auto'),
            dbc.Col(html.Div([html.Label('Themes'),
                              dcc.Dropdown(id='cat_filt',
                    options=get_cat_options(df),
                    multi=True
                    )]), width='auto'),
            dbc.Col(html.Div([html.Label('Date Range'), dcc.DatePickerRange(
                    id='date-picker-range',
                    start_date= '01/01/2010',
                    end_date= '01/01/2020'
                    # end_date_placeholder_text='Select a date!'
                    )]), width='auto'),
            html.Hr(),
            dbc.Col(html.Div([html.Label('Filters'),
                              html.Br(),
                              html.Button('Refresh Filters',
                                          id='reset_button', style={"vertical-align": "middle","border-radius": "8px",
                                                                   "background-color": "white",
                                                                    "color": "black",
                                                                    "border": "2px solid #e7e7e7"})]))
            ]),
    
        ]),
        
    ],
    style=SIDEBAR_STYLE,
)


#Nav bar for pagging

nb_pages = 10

pages = dbc.NavbarSimple(
    children=[
        dbc.Col(html.Div(html.H5("Page", style = {"color":"white"}))),
        dbc.Col(html.Div([dcc.Input(
                    id = 'num_page',
                    type= "number",
                    value = 1,
                    min = 1,
                    style = {"border-radius": "8px", "color": "black", "border": "2px solid #e7e7e7",
                            "width" : "30%",
                            "background-color": "#eee"}
                    )])),
    ],
    color="dark",
)

#Stories

def render_stories():
    list_tags = []
    for i in range(len(top_tags)-1,-1,-1):
        list_tags.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([
                    dbc.CardBody(
                        [
                            html.H5(top_tags[i][0], className="card-title"),
                            html.H6("Number of article : " + str(top_tags[i][1]), className="card-subtitle"),
                            html.P("Petite description du tag"),
                            dbc.Button("Show this tags",id = {'type': 'tags-button','index': i}),   
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
        
    
    
    return list_tags 



# Content page

card = html.Div(id ='out_card')

article = html.Div(id ='out_article')

stats = html.Div(id='out_stats')

stories = html.Div(render_stories())


app.layout = html.Div(
        [
            navbar,
            html.Hr(),
            dcc.Location(id='url', refresh=False),
            html.Div(id='page-content'),
            pages
        ]
)

index_page_layout = dbc.Row(
                            [
                                dbc.Col([sidebar], width = "100%"),
                                dbc.Col(card),
                                dbc.Col(stories,width = "30%")
                            ],
                            no_gutters=True,
                        ),

stats_page_layout = dbc.Row(
                            [
                                dbc.Col([sidebar], width = "100%"),
                                dbc.Col(stats),
                                dbc.Col(
                                            dcc.Graph(figure = fig_lang,id='graph_lang'),
                                            width={'size':3}
                                        ),
                                        dbc.Col(
                                            dcc.Graph(figure = fig_themes,id='graph_theme'),
                                            width={'size':5}
                                        )
                            ],
                            no_gutters=True,
                        ),

article_page_layout = dbc.Col(article)

stories_page_layout = dbc.Col(stories)


# Callback

# add callback for toggling the collapse on small screens
@app.callback(
    dash.dependencies.Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)

def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


@app.callback(
    dash.dependencies.Output('out_card', 'children'),
    dash.dependencies.Output('num_page','max'),
    [Input('number', 'value')],
    [Input('num_page','value')],
    [Input({'type': 'tags-button','index': ALL},'n_clicks')],
    [Input('submit-button', 'n_clicks')],
    [Input('username', 'value')],
    [State('username', 'value')],
)

def update_card_div(number,num_page,n,clicks,event,input_value):
    
    global tags_index

    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    
    filtered_df = df
    
    if "index" in changed_id:
        tags_index = int(changed_id[9])
        filtered_df = filtered_df[filtered_df["art_tag"].str.contains(top_tags[tags_index][0],case=False)]
        
    if tags_index != 11:
        filtered_df = filtered_df[filtered_df["art_tag"].str.contains(top_tags[tags_index][0],case=False)]
    
    filtered_df = filtered_df.reset_index(drop=True)
    
    element = []
    if clicks is not None or event is not None:
        for nb in range(len(filtered_df)):
            t = input_value
            if t.lower() in filtered_df['art_title'][nb].lower() or t.lower() in filtered_df['art_content'][nb].lower() or t.lower() in filtered_df['art_tag'][nb].lower():
                element.append(filtered_df['art_id'][nb])
                
    if element != []:
        filtered_df = filtered_df[filtered_df['art_id'].isin(element)]
 
    filtered_df = filtered_df.reset_index(drop=True)
    
    length_df = len(filtered_df['art_title'])
    
    if num_page*number > length_df:
        end = length_df
        filtered_df = filtered_df[((num_page*number)-number):length_df]
    else :
        filtered_df = filtered_df[((num_page*number)-number):((num_page*number))]
        end = num_page*number
    
    cards = []
    
    for i in range(((num_page*number)-number),end):
        cards.append(html.Div([
            dbc.Col(html.Div([
                dbc.Card([dbc.CardImg(src=filtered_df['src_img'][i],alt="Can't display this image",top=True),
                    dbc.CardBody(
                        [
                            dbc.CardLink(html.H5(filtered_df['art_title'][i], className="card-title"), href='/page-'+str(filtered_df['art_id'][i])),
                            html.P(filtered_df['art_content'][i][:200])
                        ]
                    )],
                    className="w-100 mb-3"
                ),
            ])
            )]))
        
    return cards,(length_df//number)+1

#Content article

@app.callback(
    Output("out_article", "children"),
    [dash.dependencies.Input('url', 'pathname')]
)

def toggle_article(pathname):
    
    index = int(pathname.split("-")[1])
    
    df_article = df[df['art_id'] == index]
    df_article = df_article.reset_index()
    
    print(df_article['art_title'][0])

    article = [html.Div([
            dbc.Card([
                dbc.CardImg(src=df_article['src_img'][0],alt="Can't display this image",top=True),
                dbc.CardBody(
                    [
                        dbc.CardLink(html.H5(df_article['art_title'][0], className="card-title"), href=df_article['art_url'][0]),
                        html.P(df_article['art_content'][0])
                    ]
                )],
                className="w-100 mb-3"
            ),
        ])
    ]
    
    print("test")

    return article


# Callback for update data in graphs
@app.callback(
    Output('graph_lang', 'figure'),
    Output('graph_theme', 'figure')
    ,
    [Input('lang_filt', 'value'),
    Input('cat_filt','value'),
    Input('date-picker-range','start_date'),
    Input('date-picker-range','end_date')
    ]
    )
def update_figure(language,theme,start,end):
    
    filtered_df_2 = df
    
    filtered_df_2['art_lang'].fillna("Unknown", inplace = True)
    # Language filter
    if language!=[]: filtered_df_2=filtered_df_2[filtered_df_2['art_lang'].isin(language)]
 
    # Filter on themes:
    if theme!=[]: filtered_df_2=filtered_df_2[filtered_df_2['src_name'].isin(theme)]
        
    #Filter on dates
#     terminal=get_min_max_dates(filtered_df_2)
#     if start!=terminal[0] or end!=terminal[1]: filtered_df_2=filtered_df_2[(filtered_df_2['art_date'] >= start) & (filtered_df_2['art_date'] <= end)]
    
    # Graph nbarticles by langue
    grp_languages=filtered_df_2.groupby('art_lang')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
    fig_lang = plot_pie_chart(filtered_df_2,'art_lang','Languages')
    fig_lang.update_layout(transition_duration=500)
    
    # Graph nbarticles by theme
    df2=filtered_df_2.groupby('src_themes')[["art_title"]].count().reset_index().rename(columns={'art_title':'nb_articles'})
    fig_themes = px.histogram(df2, x="src_themes", y='nb_articles')
    fig_themes.update_layout(transition_duration=500)
    
    return fig_lang, fig_themes

#Content    
    
@app.callback(dash.dependencies.Output('page-content', 'children'),
              [dash.dependencies.Input('url', 'pathname')])


def render_page_content(pathname):
    if pathname == "/":
        return index_page_layout
    elif pathname == "/stats":
        return stats_page_layout
    elif pathname == "/stories":
        return stories_page_layout
    else:
        return article_page_layout
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )




if __name__ == "__main__":
    app.run_server(debug=True)


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

SystemExit: 1

In [86]:
app.run_server(port=8050,host='127.0.0.1')

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run